In [1]:
# === 1. IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import optuna
import joblib
import random


import numpy as np                                 
from sklearn.model_selection import (
    train_test_split,                              
    RandomizedSearchCV, GridSearchCV,              
    cross_val_score,                               
    RepeatedKFold                                  
)
from sklearn.pipeline import Pipeline              
from sklearn.preprocessing import StandardScaler   
from sklearn.ensemble import RandomForestRegressor 
import optuna                                       
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math                                         
from sklearn.base import clone                              
from xgboost import XGBRegressor  


import numpy as np
import pandas as pd
import random, math

from sklearn.model_selection import RepeatedKFold, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

In [4]:
path = r"C:\Users\HP - PC\Downloads\Joyce's work_Adsorption-Edit.xlsx"
df = pd.read_excel(path, sheet_name="Sheet1", header=1)

# Drop first column if it's just an index / ID
df = df.drop(df.columns[0], axis=1)

# Auto-detect target column (contains "Removal" or "%")
possible_targets = [c for c in df.columns if "Removal" in str(c) or "%" in str(c)]
if not possible_targets:
    raise ValueError("Target column not found. Please set target_col manually.")
target_col = possible_targets[0]

# Ensure numeric
df[target_col] = pd.to_numeric(df[target_col], errors="coerce")
feature_cols = [c for c in df.columns if c != target_col]
for col in feature_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop missing values
df = df.dropna().reset_index(drop=True)

X = df[feature_cols].values
y = df[target_col].values

print(f"Dataset: {X.shape[0]} rows, {X.shape[1]} features. Target = {target_col}")

Dataset: 29 rows, 4 features. Target = Removal Efficiency
%


In [6]:
df

,A:pH,B:Concentration,C:Dosage\ng/100ml,D:Time,Removal Efficiency\n%
0,4.0,30,0.7,120,17.370
1,6.5,10,0.4,75,21.980
2,9.0,30,0.4,75,12.260
3,9.0,50,0.7,75,10.690
4,4.0,50,0.7,75,10.490
5,6.5,10,0.7,30,25.000
6,6.5,30,0.7,75,20.100
7,6.5,30,0.7,75,21.900
8,4.0,30,1.0,75,26.130
9,6.5,50,1.0,75,14.690


In [8]:
SEED = 43
np.random.seed(SEED)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X,                      
    y,                      
    test_size=0.20,         
    random_state=SEED,      
    shuffle=True            
)

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=SEED)

In [14]:
best_score = -np.inf
best_model = None

def objective(trial): 
    global best_score, best_model

 
    params = {
       'n_estimators': trial.suggest_int('n_estimators', 500, 10000),  
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
        'verbosity': 0,
        'tree_method': 'hist'  
    }

    


    trial_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", XGBRegressor(
            random_state=SEED,
            objective="reg:squarederror",  
            n_jobs=-1,
            **params
        ))
    ])

    
    scores = cross_val_score(trial_pipeline, X_train, y_train, cv=rkf, scoring="r2", n_jobs=-1)
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_model = trial_pipeline
        trial_pipeline.fit(X_train, y_train)
       

    return mean_score

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best R² score from CV:", best_score)
print("Best hyperparameters:", study.best_params)

[I 2025-10-22 10:28:44,718] A new study created in memory with name: no-name-1790d10a-e309-448e-bba9-5f701d086c51


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-22 10:29:28,715] Trial 0 finished with value: -1.7824959344785267 and parameters: {'n_estimators': 9343, 'learning_rate': 0.047940307986191734, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 1.2003926010338928, 'subsample': 0.6213370674407148, 'colsample_bytree': 0.9720646584564285, 'reg_alpha': 2.064380814564235, 'reg_lambda': 9.593445588822192, 'max_delta_step': 7}. Best is trial 0 with value: -1.7824959344785267.
[I 2025-10-22 10:29:45,429] Trial 1 finished with value: -3.1123764974098647 and parameters: {'n_estimators': 5363, 'learning_rate': 0.00014598906651615488, 'max_depth': 4, 'min_child_weight': 18, 'gamma': 4.355873251556532, 'subsample': 0.892553336548158, 'colsample_bytree': 0.5306986413303324, 'reg_alpha': 7.8966006878088555, 'reg_lambda': 6.071529480555108, 'max_delta_step': 6}. Best is trial 0 with value: -1.7824959344785267.
[I 2025-10-22 10:29:47,787] Trial 2 finished with value: -3.112910036118158 and parameters: {'n_estimators': 773, 'learning_rate': 0.0

In [18]:
# =========================================
# BUILD FINAL PIPELINE USING OPTUNA BEST HYPERPARAMS
# =========================================
optuna_best = study.best_trial.params

final_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", XGBRegressor(
        random_state=SEED,
        objective="reg:squarederror",
        n_jobs=-1,
        **optuna_best
    ))



    
])

final_pipeline.fit(X_train, y_train)

,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None


In [19]:
# Predict
y_train_pred = final_pipeline.predict(X_train)
y_test_pred = final_pipeline.predict(X_test)

# R² Scores
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

Train R²: 0.7687
Test R²: 0.7461
